In [274]:
import numpy as np
from scipy.stats import binom
from scipy.stats import truncnorm

In [276]:
# Parameters 

h = 1/7
pho = 2/3
gamma = 2/3
R = 19
T = 7*R

In [280]:
def truncated_normal(mean=0.5, sd=0.5, low=0, upp=1):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [278]:
#  Effective Sampling Size 

def ESS(W):                 # Takes an array as argument
    return 1/np.sum(W**2)

In [434]:
# Mutation transition

def mutation(theta, X):
    #
    X_new = X
    S, E, I, R =  X_new[:, 0], X_new[:, 1], X_new[:, 2], X_new[:, 3]
    M = theta[:16].reshape(4,4)
    betas = 1/8000 * np.dot(M, I)
    B = np.random.binomial(S, np.ones(4)-np.exp(-h*betas)) 
    C = np.random.binomial(E, 1-np.exp(-h*pho)) 
    D = np.random.binomial(I, 1-np.exp(-h*gamma)) 
    S_new = S - B
    E_new = E + B - C
    I_new = I + C - D
    R_new = R + D
    X_new[:, 0], X_new[:, 1], X_new[:, 2], X_new[:, 3] = S_new, E_new, I_new, R_new
    return X_new

In [435]:
# Likelihood of y knowing theta

def log_likelihood(y, theta, X):
    C = np.zeros((7,4))
    for i in range(7):
        X = mutation(theta, X)
        C[i] = np.random.binomial(X[:, 1], 1-np.exp(-h*pho))
    return(np.log(binom.pmf(y, np.sum(C, axis=0), theta[16:])+1.5))

In [505]:
# Initial settings
X1 = np.array([948, 0, 1, 0])
X2 = np.array([1689, 0, 1, 0])
X3 = np.array([3466, 0, 1, 0])
X4 = np.array([1894, 0, 1, 0])

def BootstrapFilter(y, theta, ESS_min, N, R=19):
    X = np.array([[X1, X2, X3, X4] for i in range(N)]) 
    w_init = np.zeros((N, 4))
    X_all = []
    X_all.append(X)
    for n in range(N):
        w_init[n] = log_likelihood(y[0], theta, X[n])
    W_init = np.zeros((N, 4))
    for n in range(N):
        W_init[n] = np.divide(w_init[n], np.sum(w_init, axis=0))
    W = W_init
    w = w_init
    X_all.append(X)
    for r in range(R):
        #print(X)
        w_hats = np.zeros((N,4))
        #print(w)
        for k in range(4):
            #print(k, W[:, k], ESS(W[:, k]))
            if ESS(W[:, k]) < ESS_min:
                A = np.random.choice(np.arange(0, N), size=N, p=W[:, k])
                w_hat = np.zeros(N)
            else:
                A = np.arange(0, N)
                w_hat = w[:, k]
            for n in range(N):
                X[n][k] = X[A[n]][k]
            w_hats[:, k] = w_hat
        for n in range(N):
            w[n] = w_hats[n]+log_likelihood(y[r], theta, X[n])
        for n in range(N):
            W[n] = np.divide(w[n], np.sum(w, axis=0))
        X_all.append(X)
    return(X_all)

In [506]:
theta = np.concatenate((np.random.gamma(5, 1, 16), truncated_normal(mean=0.5, sd=0.5, low=0, upp=1).rvs(4)), axis=None)
Y = np.array([[0,0,1,1],[0,2,6,1],[0,2,4,2],[23,73,63,11],[63,208,173,41],[73,207,171,27],[66,150,143,7],[26,40,87,29],[17,18,33,12],[3,4,13,6],[2,6,16,5],[1,6,11,3],[0,1,6,5],[0,2,2,2],[0,1,3,0],[0,1,4,6],[0,1,3,0],[2,1,7,1], [1,1,6,2]])
BootstrapFilter(Y, theta, 100, 100, R=19)

[39 91 51 48 37 30 84 46 82 50 12 90 48 48 86 57  8 31 75 53 75 64 31 28
 66  7 25 15 13 37 74 93 81 98 83 31  6 66 51  1 35 55 26 25 14 78 47 77
 14 34 31 42 22 80 84 17 94 63 77 26 52 32 35 83 72 38 59 85 88 32 60 49
 36 76 28 58 88 49 50 61 55 39 43 53 66 39 83 83 20 36 12 11 24 72 59 11
 64  0 69 46]
[19 85  1 30 28 95 57 56 23  2  0 50 73  8 21 87 69 65 23 63  0 92 64 67
  0  5 13 12 13 37 69 34 74 34 13 54 96 93 46 37 93 80 60  4 89 44 77 22
 78 20  0 43 91 39 47  0 30 38 47 51 37 69 75 43 24 57  8 89 63 50 78 88
 48 94 12 61 67 96 89 76 30 31 33 35 93 81 58 48 67 27 81 19 64 74 68 19
 91  7 38 45]
[38 57 48 89 51 42 42 98 21 67 60 53 71 37 61 36 40 63 71 67  7 60 30 39
 25 62 69 31 58 61 43 93 28 98 22 55 14 60 34  7 78 84 49 12 51 71 85 99
 62 61 65 94 11 28 61 57 36 22  7 68 13 47 95 96 96 12 33 25 48 12 75 10
 96 60 97 50 77  8 88 36 38 89 22 41 39 52 25 56 24 40 80  8 39 29 47 22
 42  6 35 96]
[80 54  2 36 98 22 19 78 65 48 31 76 53 74 35 46 46 56 41 64 71 95 80 70
 87 42 85

[array([[[   1,    1,    3,  944],
         [   2,    3,    7, 1678],
         [  47,    9,   32, 3379],
         [  11,    0,   10, 1874]],
 
        [[   0,    2,    2,  945],
         [   1,    3,    8, 1678],
         [  26,   11,   37, 3393],
         [   4,    2,   13, 1876]],
 
        [[   0,    0,    6,  943],
         [   1,    2,   11, 1676],
         [  40,    8,   37, 3382],
         [   9,    3,   11, 1872]],
 
        ...,
 
        [[   0,    0,    2,  947],
         [   4,    3,    7, 1676],
         [  26,   18,   42, 3381],
         [   4,    2,   15, 1874]],
 
        [[   1,    0,    5,  943],
         [   3,    0,    6, 1681],
         [  41,    8,   29, 3389],
         [   8,    1,    9, 1877]],
 
        [[   0,    0,    5,  944],
         [   1,    2,   11, 1676],
         [  28,    5,   28, 3406],
         [   5,    4,   10, 1876]]], shape=(100, 4, 4)),
 array([[[   1,    1,    3,  944],
         [   2,    3,    7, 1678],
         [  47,    9,   32, 3379],
   

In [ ]:
def LogL(y, theta, X):
    L = 0
    for r in range(R):
        S = 0
        for n in range(len(X[r]):
            x = X[r]
            S = S + log_likelihood(y[r], theta, x)

In [504]:
def PMMH(y, ESS_min, Nx, Nt, R=19):
    thetas = []
    theta = np.concatenate((np.random.gamma(5, 1, 16), truncated_normal(mean=0.5, sd=0.5, low=0, upp=1).rvs(4)), axis=None)
    thetas.append(theta)
    X = BootstrapFilter(y, theta, ESS_min, Nx, R=19)
    L = LogL(y, theta, X)
    for n in range(Nt):
        theta_new = np.concatenate((np.random.gamma(5, 1, 16), truncated_normal(mean=0.5, sd=0.5, low=0, upp=1).rvs(4)), axis=None)
        X_new = BootstrapFilter(y, theta_new, ESS_min, Nx, R=19)
        L_new = LogL(y, theta_new, X_new)
        v = L_new - L
        u = np.random.uniform()
        if np.log(u)<v:
            L = L_new
            thetas.append(theta_new)
        else:
            thetas.append(thetas[-1])
    return thetas